In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.naive_bayes import MultinomialNB
import re

In [ ]:
train=pd.read_csv('/kaggle/input/twitter-sentiment-analysis-hatred-speech/train.csv')

In [ ]:
test=pd.read_csv('/kaggle/input/twitter-sentiment-analysis-hatred-speech/test.csv')

In [ ]:
train.head()

In [ ]:
test.shape

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train['label'].value_counts()

In [ ]:
sns.countplot(train['label'])

# Lemmatization of the data (Data Preprocessing)

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
def normalization(tweet_list):
        lem = WordNetLemmatizer()
        normalized_tweet = []
        for word in tweet_list:
            normalized_text = lem.lemmatize(word,'v')
            normalized_tweet.append(normalized_text)
        return ' '.join(normalized_tweet)
    
train['lem_tweet']=train['tweet'].str.split().apply(normalization)
#print(normalization(tweet_list))

In [ ]:
train.head()

In [ ]:
def re_func(tweet_list):
    re_list=[]
    for word in tweet_list:
        x=re.findall('[A-Za-z]',word)
        re_list.append(''.join(x))
    
    return ' '.join(re_list)

train['lem_tweet2']=train['lem_tweet'].str.split().apply(re_func)

In [ ]:
train.drop(['tweet','lem_tweet'],axis=1,inplace=True)
train.head()

In [ ]:
X=train['lem_tweet2']
y=train['label']

# Vectorization of the features after removing the english stop words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vect=CountVectorizer(stop_words='english')
vect.fit(X,y)

In [ ]:
vect.vocabulary_

In [ ]:
def remove_digit(tweet_list):
    new_list=tweet_list
    for word in tweet_list:
        if word in ['ur','cleveland','clevelandcavaliers','im','gr','ireland','tcot','yugyoem','ig','ff','st','knicks','pa','abc','frm','mr',
                    'christina','christinagrimmie','blk','florida','gp','rb','th','pennsylvania','edwardsville','gbp','ppl','nashvilletour','dis','sta',
                   'clonakilty','doesnt','cds','ofw','rd','eos','sachintendulkar','thbihday','chrisevans','chris','evans','actor','sma','eah','id','lt',
                   'effo','aint','america','country','isnt','wimbledon','alyalshahawy','alreemgroup','ksleg','montreal','mydubai']:
            new_list.remove(word)
        
    return ' '.join(new_list)
            
train['lem_tweet3']=train['lem_tweet2'].str.split().apply(remove_digit)

In [ ]:
train.drop('lem_tweet2',axis=1,inplace=True)
train.head()

In [ ]:
X=train['lem_tweet3']
y=train['label']

In [ ]:
vect=CountVectorizer(stop_words='english')
vect.fit(X,y)
vect.vocabulary_

In [ ]:
X_trans=vect.transform(X)

In [ ]:
X_trans.shape

In [ ]:
type(X_trans)

# Model Building with multinomial naive bayes algorithm

In [ ]:
model=MultinomialNB()

model.fit(X_trans,y)


y_pred_class = model.predict(X_trans)
accuracy_score(y,y_pred_class)



In [ ]:
print(classification_report(y_pred_class,y))

# Vectorizing the test sample

In [ ]:
test.head()

In [ ]:
test['lem_tweet']=test['tweet'].str.split().apply(normalization)
test['lem_tweet2']=test['lem_tweet'].str.split().apply(re_func)

In [ ]:
X_test=test['tweet']

vect_test = CountVectorizer(vocabulary=vect.vocabulary_)
X_test_trans = vect_test.transform(X_test)



# Prediction with the test data

In [ ]:
y_pred_test=model.predict(X_test_trans)

In [ ]:
y_pred_test.shape

In [ ]:
y_pred_test_df=pd.DataFrame(y_pred_test,columns=['label'])
y_pred_test_df.head()

In [ ]:
id_df=test['id']
id_df.head()

In [ ]:
test_classification_submission=pd.concat([id_df,y_pred_test_df],axis=1)

In [ ]:
test_classification_submission.to_csv('submission_nlp.csv',index=False)